In [30]:
# Se importan las librerias a utilizar
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [31]:
# se carga el dataset
df  = pd.read_csv('./dataset/movies_limpio.csv', delimiter=',')
# se selecciona las columnas de interés
columnas_seleccionadas = ['overview', 'popularity', 'tagline', 'title']  # Reemplaza con los nombres de las columnas que deseas seleccionar
df = df[columnas_seleccionadas]
df.sample(10)

C:\Users\HP\AppData\Local\Temp\ipykernel_3596\2947862856.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df  = pd.read_csv('./dataset/movies_limpio.csv', delimiter=',')


,overview,popularity,tagline,title
39226,"A group of Internet ""sensations"" compete for a...",2.836788,NaN,Internet Famous
28110,Stelvio Massi's crime thriller centers on a dr...,0.005625,NaN,The Last Round
23759,A young man is convicted of manslaughter and s...,0.146181,NaN,Sex in Chains
44534,Interview with filmmaker Tadeusz Konwicki,0.49,NaN,Passer-By
21283,The Crash Reel tells the story of a sport and ...,2.212038,The mind has mountains,The Crash Reel
26720,Ginger Rogers is troubled when she finds her s...,0.859415,Behind this burning cross...Behind the loophol...,Storm Warning
5563,An alien creature stalks human prey.,2.218273,It Preys On Human Fear. It Feeds On Human Flesh.,Without Warning
38388,A well known actor comes to off-season Capri t...,0.191785,NaN,Il mare
42331,"A young lady, who recently lost her mother, fa...",0.131776,NaN,Lift me up
18838,A behind-the-scenes look at San Diego Comic-Co...,2.728108,NaN,Comic-Con Episode IV: A Fan's Hope


In [32]:
# Se eliminan los registros duplicados
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [33]:
#se imputan los nulos o nan con ''
resumen = df['overview'].fillna('')
frase   = df['tagline'].fillna('')

In [34]:
tfidf = TfidfVectorizer(stop_words='english') # configuracion de TfidfVectorizery, en idioma ingles

In [35]:
# se crea la una matriz TF-IDF con las puntaciones del campo overview
tfidf_matrix = tfidf.fit_transform(resumen)

In [36]:
print(tfidf_matrix)

  (0, 17747)	0.1349797800350169
  (0, 4385)	0.14747467270058714
  (0, 37995)	0.10143749487675215
  (0, 21868)	0.10436716683017426
  (0, 19622)	0.13280295914340587
  (0, 48511)	0.10337297865800553
  (0, 59468)	0.1300638381941949
  (0, 12479)	0.12542721313042463
  (0, 51059)	0.13449233434610608
  (0, 29216)	0.100975197636379
  (0, 50865)	0.09193924237792515
  (0, 39386)	0.11905314484127132
  (0, 1846)	0.14089718864405681
  (0, 58521)	0.11370452359859526
  (0, 38657)	0.2062751457819255
  (0, 9864)	0.5027729544477981
  (0, 9079)	0.10633362289117387
  (0, 7486)	0.1237882025909978
  (0, 56822)	0.11132624558524169
  (0, 28707)	0.1330993857684982
  (0, 38975)	0.0872198441251726
  (0, 67816)	0.14876952353745504
  (0, 3157)	0.4117381439416864
  (0, 73407)	0.48094829735119926
  (0, 38053)	0.10741684376115744
  :	:
  (45377, 26935)	0.07349389879443743
  (45377, 18901)	0.09270286653454458
  (45377, 18101)	0.07466750517866716
  (45377, 38975)	0.06832348860473762
  (45378, 16506)	0.3236733741932533
 

In [37]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix) # se calcula la matriz de coseno

In [39]:
def get_recommendations(df, column, value, cosine_similarities, limit=5):
    """Return a dataframe of content recommendations based on TF-IDF cosine similarity.
    
    Args:
        df (object): Pandas dataframe containing the text data. 
        column (string): Name of column used, i.e. 'title'. 
        value (string): Name of title to get recommendations for, i.e. 1982 Ferrari 308 GTSi For Sale by Auction
        cosine_similarities (array): Cosine similarities matrix from linear_kernel
        limit (int, optional): Optional limit on number of recommendations to return. 
        
    Returns: 
        Pandas dataframe. 
    """
    
    # Return indices for the target dataframe column and drop any duplicates
    indices = pd.Series(df.index, index=df[column]).drop_duplicates()

    # Get the index for the target value
    target_index = indices[value]

    # Get the cosine similarity scores for the target value
    cosine_similarity_scores = list(enumerate(cosine_similarities[target_index]))

    # Sort the cosine similarities in order of closest similarity
    cosine_similarity_scores = sorted(cosine_similarity_scores, key=lambda x: x[1], reverse=True)

    # Return tuple of the requested closest scores excluding the target item and index
    cosine_similarity_scores = cosine_similarity_scores[1:limit+1]

    # Extract the tuple values
    index = (x[0] for x in cosine_similarity_scores)
    scores = (x[1] for x in cosine_similarity_scores)    

    # Get the indices for the closest items
    recommendation_indices = [i[0] for i in cosine_similarity_scores]

    # Get the actutal recommendations
    recommendations = df[column].iloc[recommendation_indices]

    # Return a dataframe
    df = pd.DataFrame(list(zip(index, recommendations, scores)), 
                      columns=['index','recommendation', 'cosine_similarity_score']) 

    return df

In [44]:
recommendations = get_recommendations(df, 
                                      'title', 
                                      'Toy Story', 
                                      cosine_similarities)
recommendations.head(5)

,index,recommendation,cosine_similarity_score
0,15341,Toy Story 3,0.531863
1,2995,Toy Story 2,0.471826
2,10296,The 40 Year Old Virgin,0.275001
3,24495,Small Fry,0.273041
4,23817,Andy Hardy's Blonde Trouble,0.235456
